In [0]:
%sql
SELECT a.account_id,
	a.policy_number,
	a.policy_state,
	c.lob,
	a.product_code,
	a.program,
	a.organization,
	a.marketing_plan,
	b.book,
	CASE
		WHEN a.organization = 'Unknown' THEN x.organization
		ELSE a.organization
	END AS final_organization,
	CASE
		WHEN a.organization = 'Unknown' THEN xb.book
		ELSE b.book
	END AS final_book
FROM ventiv.dbo.policy_import_01 AS a
LEFT JOIN ventiv.dbo.ref_organization AS b ON
		a.organization = b.organization
LEFT JOIN ventiv.dbo.ref_lob AS c ON
		a.primary_coverage = c.coverage
LEFT JOIN ventiv.dbo.xref_marketing_plan_to_organization AS x ON
		LEFT(a.marketing_plan, 3) = x.marketing_plan_code
LEFT JOIN ventiv.dbo.ref_organization AS xb ON
		x.organization = xb.organization
WHERE 1 = 1
	AND a.program = 'EdgePro'
	AND a.organization = 'Unknown'
ORDER BY
	marketing_plan;

In [0]:
%sql
UPDATE a
SET a.organization = CASE
		WHEN a.organization = 'Unknown' THEN x.organization
		ELSE a.organization
	END
FROM ventiv.dbo.policy_import_01 a
LEFT JOIN ventiv.dbo.ref_organization b  ON
	a.organization = b.organization
LEFT JOIN ventiv.dbo.ref_lob c  ON
	a.primary_coverage = c.coverage
LEFT JOIN ventiv.dbo.xref_marketing_plan_to_organization x  ON
	LEFT(a.marketing_plan, 3) = x.marketing_plan_code
LEFT JOIN ventiv.dbo.ref_organization xb  ON
	x.organization = xb.organization
WHERE 1 = 1
	AND a.program = 'EdgePro'
	AND a.organization = 'Unknown';

In [0]:
df_patch_query_sql = spark.sql(f"""
    SELECT a.account_id,
	a.policy_number,
	a.policy_state,
	c.lob,
	a.product_code,
	a.program,
	a.organization,
	a.marketing_plan,
	b.book,
	CASE
		WHEN a.organization = 'Unknown' THEN x.organization
		ELSE a.organization
	END AS final_organization,
	CASE
		WHEN a.organization = 'Unknown' THEN xb.book
		ELSE b.book
	END AS final_book
    FROM df_import_01 AS a
    LEFT JOIN delta_ref_org AS b ON
            a.organization = b.organization
    LEFT JOIN delta_ref_lob AS c ON
            a.primary_coverage = c.coverage
    LEFT JOIN delta_xref_marketing_plan_to_organization AS x ON
            LEFT(a.marketing_plan, 3) = x.marketing_plan_code
    LEFT JOIN delta_ref_org AS xb ON
            x.organization = xb.organization
    WHERE 1 = 1
        AND a.program = 'EdgePro'
        AND a.organization = 'Unknown'
    ORDER BY
        marketing_plan;
""")

df_patch_query_sql.display()
# print(df_patch_query_sql.count())

In [0]:
SELECT '{as_of_date}' AS as_of,
    b.book,
    b.account_name,
    CASE WHEN a.billing_code in ('161856')
            THEN 'Tree Pro'
        ELSE b.specialty_program
    END AS specialty_program,
    b.carrier,	
    CASE WHEN a.primary_coverage IN ('EPL/POL')
            THEN a.primary_coverage
        ELSE c.lob 
    END AS LOB, /* Not adding 'EPL/POL' as an entry to the reference LOB table because claims should NOT come in this way; these premium records are ok. */
    a.*,
    (CAST(a.sub_producer_commission_char AS FLOAT) / 100) AS sub_producer_commission
FROM df_import_01 AS a
    LEFT JOIN delta_ref_org AS b ON
        a.organization = b.organization
    LEFT JOIN delta_ref_lob AS c ON
        a.primary_coverage = c.coverage;